## Question 2 ##
## Texture synthesis ##
I used the exact same method described in lectures(12th). Meaning that I started from a random patch(100*100) in the texture and then
continued by filling the first row by finding a patch which shared a vertical strip with the initial one. In order to find the best match for this patch, I used the
default template match function in cv2. Then I continued to find the best min cut which its left side is from the pixels of the left patch and its right side is from the right one.
The general algorithm for when we are not in the first row is mostly the same. except thta the patches should be found horizontally and the min cut is also horizontally when we are at the first column.
Moreover, in the general case, We use the L shape so that the next patch to be found share the top and left side of it to the previous ones.

In [ ]:
import cv2
import numpy as np
import random


def synthesize(image, res_name):
    # first choose a random patch:
    texture = image
    height, width, _ = texture.shape
    patch_size, strip_size, diff = 100, 20, 80
    h_start_index, w_start_index = random.randint(0, height - patch_size), random.randint(0, width - patch_size)
    patch = texture[h_start_index: h_start_index + patch_size, w_start_index: w_start_index + patch_size, :]

    result = np.zeros((2500, 2500, 3))
    result[0: patch_size, 0: patch_size, :] = patch

    for i in range(0, 31):
        for j in range(0, 31):
            if i == 0 and j != 0:
                adjacent_v_patch = result[0: patch_size, j * diff + strip_size - patch_size: j * diff + strip_size, :]
                new_patch = matchPatch(texture, adjacent_v_patch, strip_size, patch_size, direction=0)
                min_cut_strip = findMinCutPatch(adjacent_v_patch, new_patch, strip_size, direction=0)
                result[0:patch_size, j * diff: j * diff + strip_size, :] = min_cut_strip
                result[0:patch_size, j * diff + strip_size: j * diff + patch_size, :] = new_patch[:, strip_size:, :]
            if i != 0 and j == 0:
                adjacent_h_patch = result[i * diff + strip_size - patch_size: i * diff + strip_size, 0: patch_size, :]
                new_patch = matchPatch(texture, adjacent_h_patch, strip_size, patch_size, direction=1)
                min_cut_strip = findMinCutPatch(adjacent_h_patch, new_patch, strip_size, direction=1)
                result[i * diff: i * diff + strip_size, 0:patch_size, :] = min_cut_strip
                result[i * diff + strip_size: i * diff + patch_size, 0:patch_size, :] = new_patch[strip_size:, :, :]
            elif i != 0 and j != 0:
                adjacent_tc_patch = result[diff * i: diff * i + patch_size, diff * j: diff * j + patch_size, :]
                new_patch = matchPatch(texture, adjacent_tc_patch, strip_size, patch_size, direction=2)
                min_cut_patch = findMinCutPatch(adjacent_tc_patch, new_patch, strip_size, direction=2)
                result[diff * i: diff * i + patch_size, diff * j: diff * j + patch_size, :] = min_cut_patch

    res = np.zeros((2500, 2500 + texture.shape[1] + 300))
    res[:,:2500,:] = result
    res[300:texture.shape[0], 2500 + 100 + texture.shape[1], :] = texture
    cv2.imwrite("../results/" + res_name + ".jpg", result)

As described above, I used the inbuilt function in cv2 to find the best patch. However,
I don't just go for one best match, instead I choose from some of the best ones ranomly and
the way that it is done is by the help of findRandomMin function which
sorts the output x and y locations and then checks if they are valid (since the strip is a part of the next patch which is to
be selected hence we should consider the valid coordinates for that).

In [ ]:
def matchPatch(texture, patch, strip_size, patch_size, direction=0):
    texture, patch = texture.astype(np.float32), patch.astype(np.float32)
    mask, limit_y, limit_x = np.zeros(patch.shape, dtype=np.float32), None, None
    if direction == 0:
        result, limit_x = cv2.matchTemplate(texture, patch[:, -strip_size:, :], 1), texture.shape[1] - patch_size
    elif direction == 1:
        result, limit_y = cv2.matchTemplate(texture, patch[-strip_size:, :, :], 1), texture.shape[0] - patch_size
    else:
        mask[0:strip_size, 0:strip_size, :] = 1
        result, limit_y = cv2.matchTemplate(texture, patch, 1, mask=mask), texture.shape[0]
    ry, rx = findRandomMin(result, 5, limit_y, limit_x)
    return texture[ry:ry + patch_size, rx:rx + patch_size, :]

In [ ]:
def findRandomMin(result, num, limit_y, limit_x):
    idx = result.ravel().argsort()[:num]
    locations = np.stack(np.unravel_index(idx, result.shape)).T
    if limit_y is not None:
        indices = np.where(locations[:, 0] < limit_y)
    elif limit_x is not None:
        indices = np.where(locations[:, 1] < limit_x)
    yLoc, xLoc = np.zeros(indices[0].size, int), np.zeros(indices[0].size, int)
    for i in range(yLoc.shape[0]):
        yLoc[i], xLoc[i] = locations[indices[0][i], 0], locations[indices[0][i], 1]
    if yLoc.shape[0] == 0:
        return findRandomMin(result, num + 1, limit_y, limit_x)
    else:
        r = np.random.randint(0, yLoc.shape[0])
        return yLoc[r], xLoc[r]

finding the min cut is separated for each of the directions (horizontally, vertically and both).
They all use the same function with minor changes. In the horizontal direction, we simply get the mask array(array of 0s and 1s)
from makeMinCutHorizontallyMask function and apply it to the strips. It is the same fot vertical direction
except that I rotated the strips 90 degrees counter clock wise and then again rotate the result 90deg clockwise.
For the L shape direction, I used the mask for top and left strips as above and then calculated the OR of these arrays.
Finally, applied the masks to the strips and replaced the strip in the targeted place.

In [ ]:
def findMinCutPatch(patch1, patch2, strip_size, direction=0):
    if direction == 1:
        strip1, strip2 = patch1[-strip_size:, :, :], patch2[:strip_size, :, :]
        mask = makeMinCutHorizontallyMask(strip1, strip2)
        return strip1 * mask + strip2 * (1 - mask)
    elif direction == 0:
        strip1, strip2 = patch1[:, -strip_size:, :], patch2[:, :strip_size, :]
        mask = np.rot90(makeMinCutHorizontallyMask(np.rot90(strip1, -1), np.rot90(strip2, -1)), 1)
        return strip1 * mask + strip2 * (1 - mask)
    else:
        mask_h, mask_v = np.zeros(patch2.shape), np.zeros(patch2.shape)
        strip1, strip2 = patch1[:strip_size, :, :], patch2[:strip_size, :, :]
        mask_h[:strip_size, :, :] = makeMinCutHorizontallyMask(strip1, strip2)
        strip3, strip4 = patch1[:, :strip_size, :], patch2[:, :strip_size, :]
        mask_v[:, :strip_size, :] = np.rot90(makeMinCutHorizontallyMask(np.rot90(strip3, -1), np.rot90(strip4, -1)), 1)
        mask = np.logical_or(mask_v, mask_h)
        return patch1 * mask + patch2 * (1 - mask)

The details of finding the min cut mask are the same as the lecture notes.
I first calculated the difference array of two strips and then made a cost and path arrays.
the first column of cost array contains the same values as the first column of difference array.
for each of the elements in the next column I find the best element is the left side colomn which minimize the cost and then
store the neighbor in the path array. I continue this where I finally reach the last column. find the min value in it (it should say
which row to start in the last column) and then use the path array to find our which row to go in the next column.


In [ ]:
def makeMinCutHorizontallyMask(strip1, strip2):
    difference = np.sum(np.abs(strip1 - strip2), axis=2)

    cost, path = np.zeros(difference.shape, dtype=np.int16), np.zeros(difference.shape, dtype=np.int16)
    cost[:, 0] = difference[:, 0]

    for j in range(1, difference.shape[1]):
        for i in range(difference.shape[0]):
            min_val, res_neighbor = 100000000, 0
            for neighbor in -1, 0, 1:
                if 0 <= i - neighbor < difference.shape[0]:
                    if cost[i - neighbor, j - 1] < min_val:
                        min_val, res_neighbor = cost[i - neighbor, j - 1], neighbor
            cost[i, j] = min_val + difference[i, j]
            path[i, j - 1] = i - res_neighbor
    min_i = (np.where(cost[:, cost.shape[1] - 1] == np.amin(cost[:, cost.shape[1] - 1])))[0][0]
    mask = np.zeros(strip1.shape)

    for col in range(path.shape[1] - 1, 0, -1):
        mask[:min_i, col, :] = 1
        min_i = path[min_i, col - 1]
    return mask